<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
       <img src="https://www.uoc.edu/content/experience-fragments/uoc-common/ww/ca/site/header/master/_jcr_content/root/container/uocheader/image.coreimg.png/1707386740074/logo.png", align="left">
    </div>
</div>

<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;">M1.771 · Privacidad</p>
    <p style="margin: 0; text-align:right;">Máster Universitario en Ciberseguridad y Privacidad</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>

</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

Privacidad
============================

--- 

PEC2 - Actividad 3 con datos de Lightbeam
-----------------------------------------------------

Este *notebook* es una herramienta para responder a las preguntas planteadas en
la activitad 3 de la PEC2.

### Introducción
El conjunto de datos con el que vamos a trabajar (`ThunderbeamData.json`)
consta de las páginas registradas con Lightbeam, la extensión del navegador
que utiliza visualizaciones interactivas para mostrarnos las relaciones entre 
terceros y las páginas que visitamos. 

In [2]:
import pandas as pd
import json

# Generamos un dataframe con los datos que hemos descargado del Lightbeam.
# JN Puesto que el formato no es compatible con pd.read_json, lo cargamos primero con json.load() y creamos el df a medida
# df = pd.read_json('lightbeamDataPEC2.json')
# df = df.transpose()

with open('lightbeamDataPEC2.json', "r", encoding="utf-8") as f:
    data = json.load(f)

websites = data.get("websites", {})
rows = []
for domain, info in websites.items():
    row = {
        "hostname": info.get("hostname"),
        "firstParty": info.get("firstParty", False),
        "firstPartyHostnames": info.get("firstPartyHostnames", []),
        "thirdParties": info.get("thirdParties", [])
    }
    rows.append(row)
df = pd.DataFrame(rows, columns=["hostname", "firstParty", "firstPartyHostnames", "thirdParties"])

# # Restringimos los registros a las páginas que hemos visitado.
pagina = df[df.firstParty == True]

In [3]:
# Imprimimos las columnas del dataframe que hemos cargado.
print(pagina.columns.values)

# Imprimimos la lista de páginas que hemos visitado.
print(pagina.hostname)

['hostname' 'firstParty' 'firstPartyHostnames' 'thirdParties']
53           www.amazon.es
55    www.elcorteingles.es
60       www.mediamarkt.es
61       www.microsoft.com
62            www.omya.com
63             www.uoc.edu
Name: hostname, dtype: object


In [4]:
# Generamos las parejas de páginas, la lista de terceros en común, y cuántos son.
PaginesTercers = [[pagina.hostname[u], 
                   pagina.hostname[v], 
                   list(set(pagina.thirdParties[u]) & set(pagina.thirdParties[v])), 
                   len(list(set(pagina.thirdParties[u]) & set(pagina.thirdParties[v])))]
                  for u in pagina.index 
                  for v in pagina.index
                  if u != v]

# Convertimos los datos que hemos calculado en un dataframe.
PaginesTercers = pd.DataFrame(PaginesTercers, 
                              columns = ['Pagina1', 
                                         'Pagina2', 
                                         'TercersEnComu', 
                                         'Interseccio'])  

Pregunta 1 - Número total de terceros

In [18]:
# JN Calculamos el número de terceros conectados iteramos por data["websites"] y descartamos las páginas que 
#    hemos visitado intencionadamente ("firstParty" = true)
all_thirdParties = set()

for domain, info in data["websites"].items():
    if not info["firstParty"]:
        # Es un dominio de tercero
        all_thirdParties.add(domain)

print("Número total de terceros:", len(all_thirdParties))

Número total de terceros: 59


Pregunta 2 - Página con más terceros, y cuántos tiene

In [16]:
# Aseguramos que 'pagina' es una copia independiente del DF para evitar warnings
pagina = pagina.copy()

# Añadimos una nueva columna con el número de terceros por página
pagina["num_terceros"] = pagina["thirdParties"].apply(len)

# Buscamos la página con el mayor número de terceros
pagina_max_terceros = pagina.loc[pagina["num_terceros"].idxmax(), ["hostname", "num_terceros"]]

print(f"Página con más terceros: {pagina_max_terceros['hostname']} ({pagina_max_terceros['num_terceros']} terceros)")


Página con más terceros: www.mediamarkt.es (22 terceros)


Pregunta 3 - Las 2 páginas con más terceros en común, y cuántos son

In [10]:
# Calculamos cuáles son las páginas que tienen más terceros en común.
PaginesTercers.iloc[PaginesTercers.Interseccio.idxmax()]


Pagina1                                          www.mediamarkt.es
Pagina2                                          www.microsoft.com
TercersEnComu    [bat.bing.net, www.facebook.com, www.googletag...
Interseccio                                                      6
Name: 12, dtype: object

In [11]:
PaginesTercers


,Pagina1,Pagina2,TercersEnComu,Interseccio
0,www.amazon.es,www.elcorteingles.es,[],0
1,www.amazon.es,www.mediamarkt.es,[],0
2,www.amazon.es,www.microsoft.com,[],0
3,www.amazon.es,www.omya.com,[],0
4,www.amazon.es,www.uoc.edu,[],0
5,www.elcorteingles.es,www.amazon.es,[],0
6,www.elcorteingles.es,www.mediamarkt.es,[www.googletagmanager.com],1
7,www.elcorteingles.es,www.microsoft.com,[www.googletagmanager.com],1
8,www.elcorteingles.es,www.omya.com,"[www.googletagmanager.com, assets.adobedtm.com]",2
9,www.elcorteingles.es,www.uoc.edu,"[geolocation.onetrust.com, www.googletagmanage...",3


Pregunta 4 - Tercero más frecuente

In [17]:
# Descomponemos la lista de terceros en filas individuales
df_terceros = pagina.explode("thirdParties")[["thirdParties"]]

# Contamos en cuántas páginas aparece cada tercero (frecuencia de terceros únicos)
tercero_frecuente = df_terceros["thirdParties"].value_counts().idxmax()
num_apariciones = df_terceros["thirdParties"].value_counts().max()

# Mostramos el resultado
print(f"El tercero más frecuente es: {tercero_frecuente}, presente en {num_apariciones} páginas distintas.")


El tercero más frecuente es: www.googletagmanager.com, presente en 5 páginas distintas.
